Question 1 PART 1: (1 point)

Load the MovieData.csv dataset into a pandas DataFrame as described in this week's lesson, and use it to find the following values:

a. What is the median profit of movies with budgets of over $50M?

b. How many movies were released by each film distributor? Output the results to a csv file.

PART 2: (6 points)

c. What are the mean and median movie profits by decade? Which decade was the most profitable? (Hint: Answering this question requires several steps: grouping the movies by decade, computing the mean and median profits for each decade, and combining the results back together.)

In [1]:
import pandas as pd
import datetime as dt

def makedate(str):
    m,d,y = str.split("/")
    m = int(m)
    d = int(d)
    y = int(y)
    if y > 13:
        y += 1900
    else:
        y += 2000
    return dt.datetime(y,m,d)
movies = pd.read_csv("MovieData.csv", na_values = ["Unknown", "Unkno"], sep='\t', parse_dates = [0], date_parser = makedate)
movies = movies.fillna(0)
movies["Worldwide Gross"][movies["Worldwide Gross"] == 0] = movies["US Gross"]
movies["profit"] = movies["Worldwide Gross"] - movies["Budget"]
print("the median profit of movies with budgets of over $50M = ", movies[movies["Budget"] > 50e6].profit.median())

by_distributor = movies.groupby("Distributor")
count = by_distributor.Movie.aggregate(len)
print(count)
print("movies count released by each film distributor is written to file moviecount.csv")
count.to_csv("moviecount.csv")#output movie counts by each film distributor to csv file

movies["year"] = movies.Release_Date.apply(lambda x : x.year)
movies["decade"] = movies.year.apply(lambda x : int(x / 10)*10)
by_decade = movies.groupby("decade")
decade_means = by_decade.profit.mean()
decade_median = by_decade.profit.median()
df = pd.DataFrame({'mean_profit': decade_means,'median_profit':decade_median})
print(df)
print("decade",decade_means.idxmax(),  "was the most profitable decade")


the median profit of movies with budgets of over $50M =  89246220.0
Distributor
0                                  659
20th Century Fox                   230
3D Entertainment                     1
8 X Entertainment                    1
ART                                  1
Access                               1
Alliance                             4
American International Pictures      1
Anchor Bay                           4
Apparition                           4
Artisan                             23
Artistic License                     1
Atlantic                             1
Attitude Films                       1
Avatar                               1
Avco Embassy                         5
Barking Cow                          1
Big Pictures                         1
Bigger Picture                       1
Black Diamond Pictures               1
Buena Vista                        227
CBS Films                            3
CFP                                  1
CHRIST                 

C:\Jenny\DataScience\anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


Question 2 PART 1: (1 point)

Load the earthquake data in QuakeData.csv into a DataFrame, and use it to answer the following questions:

a. What is the median earthquake magnitude?

b. What is the correlation between magnitude and depth?

PART 2: (7 points)

c. What fraction (not count) of earthquakes happen each month, across all years (i.e. all earthquakes occurring in January as a proportion of the grand total, all earthquakes in February as a proportion of the grand total, etc.)?

d. Is there correlation between the number of movies released monthly (i.e. Jan-1990, Feb-1990... ) , and the number of earthquakes in that month?

In [2]:
def makeDate(str):
    y = int(str[0:4])
    m = int(str[5:7])
    d = int(str[8:10])
    h = int(str[11:13])
    min = int(str[14:16])
    s = int(str[17:19])
    return dt.datetime(y, m, d, h, min, s)
quakes = pd.read_csv("QuakeData.csv", sep=",", parse_dates = [0], date_parser = makeDate)
print("the median earthquake magnitude = ", quakes.Magnitude.median())
print("the correlation between magnitude and depth = ", quakes.Magnitude.corr(quakes.Depth))

the median earthquake magnitude =  4.5
the correlation between magnitude and depth =  0.029175159159976656


In [3]:
quakes['month'] = quakes.DateTime.apply(lambda x: x.month)
quakes_by_month = quakes.groupby('month')
counts = quakes_by_month.EventID.aggregate(len)
df = pd.DataFrame({'quakecounts': counts})
total = df.quakecounts.sum()
df['fraction'] = df['quakecounts']/total
print(df)

quakes['year'] = quakes.DateTime.apply(lambda x: x.year)
movies['month'] = movies.Release_Date.apply(lambda x : x.month)

movies_by_monthly = movies.groupby(['year','month'])
quakes_by_monthly = quakes.groupby(['year','month'])
moviecounts = movies_by_monthly.Movie.aggregate(len)
quakescounts = quakes_by_monthly.EventID.aggregate(len)
quakemovie_df = pd.DataFrame({'quakecounts': quakescounts, 'moviecounts': moviecounts})
corr = quakemovie_df.quakecounts.corr(quakemovie_df.moviecounts)
print("the number of movies released monthly and the number of earthquakes in that month correlation=",corr)

       quakecounts  fraction
month                       
1             1024  0.080732
2             1081  0.085225
3             1145  0.090271
4             1393  0.109823
5             1058  0.083412
6              900  0.070956
7              882  0.069536
8             1022  0.080574
9             1132  0.089246
10            1051  0.082860
11             996  0.078524
12            1000  0.078839
the number of movies released monthly and the number of earthquakes in that month correlation= 0.016073604515738776


Not much correlation between the number of movies released monthly and the number of earthquakes in that month